In [ ]:
import requests
from bs4 import BeautifulSoup as bs 
import warnings as wgs
wgs.filterwarnings('ignore')
import numpy as np

In [ ]:
URL=[]
for i in range(1,11):
    URL.append(requests.get('https://www.amazon.fr/JBL-TUNE-510BT-supra-auriculaire-multi-points/product-reviews/B08VD6SRBZ/ref=cm_cr_getr_d_paging_btm_prev_{}?ie=UTF8&reviewerType=all_reviews&pageNumber={}'.format(i,i),headers = {
'authority': 'www.amazon.com',
'pragma': 'no-cache',
'cache-control': 'no-cache',
'dnt': '1',
'upgrade-insecure-requests': '1',
'user-agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
'sec-fetch-site': 'none',
'sec-fetch-mode': 'navigate',
'sec-fetch-dest': 'document',
'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
}))
URL

In [ ]:
Page=[]
for url in URL:
    Page.append(bs(url.text,'html.parser'))
len(Page)

In [ ]:
M=[]
N=[]
for i in Page:
    M.append(i.find_all('span', attrs={'class': 'a-icon-alt'}))

for i in range(len(M)):
    u=M[i]
    for j in u:
        N.append(j.text.split()[0])
N



In [ ]:
M1=[]
N1=[]
for i in Page:
    M1.append(i.find_all("span",{'data-hook':"review-body"}))


for i in range(len(M1)):
    u=M1[i]
    for j in u:
        N1.append(j.text.strip().split('\n'))
N1



In [ ]:
Page[4].find_all('span', attrs={'class': 'a-icon-alt'})

In [ ]:
import pandas as pd
D1=pd.DataFrame(N1)
D1['commentaires']=D1[0]
D1.drop(0,axis=1)

In [ ]:
import pandas as pd
D2=pd.DataFrame(N)
D2['sentiment']=D2[0].str.replace(',','.').str.replace('5.0','Positive').str.replace('4.0','Neutre').str.replace('3.0','Neutre').str.replace('2.0','Négative').str.replace('1.0','Négative')
D2.drop(0,axis=1,inplace=True)
D2

In [ ]:
data=pd.concat([D1,D2[0:100]],axis=1)
data.drop(0,axis=1,inplace=True)
data.dropna()


In [ ]:
data_positif = data[data['sentiment']=='Positive']
data_neutre = data[data['sentiment']=='Neutre']

dimb = pd.concat([data_positif,data_neutre])
dimb.value_counts(['sentiment'])
print(dimb)

In [ ]:
pip install imblearn


In [ ]:
from imblearn.under_sampling import  RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
dbal, dbal['sentiment']=rus.fit_resample(dimb[['commentaires']],
                                                           dimb['sentiment'])
dbal


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dbal, test_size=0.33, random_state=42)

In [ ]:
x_train, y_train = train['commentaires'], train['sentiment']
x_test, y_test = test['commentaires'], test['sentiment']
x_test, y_test

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(x_train)
# also fit the test_x_vector
test_x_vector = tfidf.transform(x_test)
train_x_vector

In [ ]:
pd.DataFrame.sparse.from_spmatrix(train_x_vector,
                                  index=x_train.index,
                                  columns=tfidf.get_feature_names())

In [ ]:
from sklearn.svm import SVC

s_v_c = SVC(kernel='linear')
s_v_c.fit(train_x_vector,y_train)

In [ ]:
from sklearn.metrics import plot_confusion_matrix,confusion_matrix

plot_confusion_matrix(s_v_c,test_x_vector,y_test)

In [ ]:
TN, FP, FN, TP = confusion_matrix(y_test,  s_v_c.predict(test_x_vector)).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP+TN) /(TP+FP+TN+FN)

print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'C': np.linspace(0,3,num=10),'kernel':['linear', 'rbf']}
s_v_c = SVC()

s_v_c_grid = GridSearchCV(s_v_c,parameters, cv=3)

s_v_c_grid.fit(train_x_vector, y_train)

In [ ]:
print(s_v_c_grid.best_params_)
print(s_v_c_grid.best_estimator_)

In [ ]:
svc =SVC(C=0.3333333333333333, kernel='linear')
svc.fit(train_x_vector,y_train)

In [ ]:
plot_confusion_matrix(s_v_c1,test_x_vector,y_test)

In [ ]:
TN, FP, FN, TP = confusion_matrix(y_test, s_v_c1.predict(test_x_vector)).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP+TN) /(TP+FP+TN+FN)

print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, 
                            s_v_c1.predict(test_x_vector),
                            labels=['positive', 'Neutre']))


In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(train_x_vector, y_train)


In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(logreg,test_x_vector,y_test)

In [ ]:
TN, FP, FN, TP = confusion_matrix(y_test,  logreg.predict(test_x_vector)).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP+TN) /(TP+FP+TN+FN)

print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, 
                            logreg.predict(test_x_vector),
                            labels=['positive', 'Neutre']))


In [ ]:
from sklearn.model_selection import GridSearchCV

grid={"C":np.logspace(-10,10,4), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg1 = LogisticRegression()
logreg1_grid = GridSearchCV(logreg1,grid, cv=5)
logreg1_grid.fit(train_x_vector, y_train)

In [ ]:
print(logreg1_grid.best_params_)
print(logreg1_grid.best_estimator_)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg1 =LogisticRegression(C=1e-10)

logreg1.fit(train_x_vector, y_train)


In [ ]:
plot_confusion_matrix(logreg1,test_x_vector,y_test)

In [ ]:
TN, FP, FN, TP = confusion_matrix(y_test,  logreg1.predict(test_x_vector)).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

accuracy =  (TP+TN) /(TP+FP+TN+FN)

print('Accuracy of the binary classification = {:0.3f}'.format(accuracy))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, 
                            logreg1.predict(test_x_vector),
                            labels=['positive', 'Neutre']))
